In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

# Example URL (replace with real one)
url = 'https://example.gov/new-llc-listings'

# Fetch page
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Parse listings
listings = []
for row in soup.select('table tr'):
    cols = row.find_all('td')
    if len(cols) >= 3:
        listings.append({
            'business_name': cols[0].text.strip(),
            'filing_date': cols[1].text.strip(),
            'location': cols[2].text.strip()
        })

# Save to SQLite DB
conn = sqlite3.connect('llc_listings.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS llcs (
    id INTEGER PRIMARY KEY,
    business_name TEXT,
    filing_date TEXT,
    location TEXT
)''')

for entry in listings:
    cursor.execute('INSERT INTO llcs (business_name, filing_date, location) VALUES (?, ?, ?)',
                   (entry['business_name'], entry['filing_date'], entry['location']))

conn.commit()
conn.close()
